<a href="https://colab.research.google.com/github/aquafire088/invoice-processor/blob/main/invoices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch pillow python-multipart
!pip install flask-ngrok pyngrok  # For web interface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
# Install required packages in Colab
!pip install fastapi uvicorn python-multipart nest-asyncio pyngrok
!pip install Pillow pdf2image
!apt-get update && apt-get install -y poppler-utils

In [1]:
!pip install fastapi uvicorn pyngrok python-multipart nest_asyncio -q

In [2]:
!pip install -q transformers accelerate datasets timm pdf2image sentencepiece
!apt-get install -y poppler-utils  # Needed for PDF to image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove a

In [1]:
from transformers import (
    AutoTokenizer,
    AutoProcessor,
    AutoModelForImageTextToText,
    AutoConfig
)
import torch

model_path = "nanonets/Nanonets-OCR-s"

# Load config and disable FlashAttention 2
config = AutoConfig.from_pretrained(model_path)
config._flash_attn_2_enabled = False

# Load model
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    config=config,
    torch_dtype="auto",
    device_map="auto"
)
model.eval()

# Tokenizer + Processor
tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("✅ Nanonets model loaded on", device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


video_preprocessor_config.json: 0.00B [00:00, ?B/s]

✅ Nanonets model loaded on cuda


In [ ]:
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText, AutoConfig
import os
import io
import base64
import uuid
import json
import shutil
import tempfile
from pathlib import Path
from typing import List, Optional, Dict, Any
from datetime import datetime
import asyncio
import nest_asyncio
from threading import Thread
import time
import torch # Import torch here

# FastAPI imports
from fastapi import FastAPI, File, UploadFile, Form, HTTPException, Request
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

# Image processing
from PIL import Image
import pdf2image

In [ ]:
model_path = "nanonets/Nanonets-OCR-s"

# Disable FlashAttention 2
config = AutoConfig.from_pretrained(model_path)
config._flash_attn_2_enabled = False

# ✅ Do NOT enable flash_attention_2 here
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    config=config,
    torch_dtype="auto",
    device_map="auto"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


video_preprocessor_config.json: 0.00B [00:00, ?B/s]

In [3]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import uvicorn, nest_asyncio
from PIL import Image
import io


app = FastAPI()
@app.post("/process")
async def process(file: UploadFile = File(...), prompt: str = Form(...)):
    contents = await file.read()
    image = Image.open(io.BytesIO(contents)).convert("RGB")

    print(f"📎 File: {file.filename}, Prompt: {prompt}")

    # Preprocess
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

    # Generate
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=512)
    output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return {
        "fileName": file.filename,
        "promptUsed": prompt,
        "extractedFields": output,
        "rawResponse": {"model": "nanonets/Nanonets-OCR-s"}
    }

!ngrok config add-authtoken "2zWS1nqfKyYmG4UWJy3NjWFagV2_GJc4qnVUibgpBhQdguhg"
# Start ngrok on a different port if needed
port = 8000
public_url = ngrok.connect(port)
print("🚀 Public Colab endpoint:", public_url)

nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=port)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Public Colab endpoint: NgrokTunnel: "https://9c5f-34-145-65-140.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [3438]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


📎 File: 94846[1]_page-0001.jpg, Prompt: Extract the following fields from the invoice: Vendor Name, Invoice Number.
INFO:     196.117.160.61:0 - "POST /process HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",

In [8]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import uvicorn, nest_asyncio

app = FastAPI()

@app.post("/process")
async def process(file: UploadFile = File(...), prompt: str = Form(...)):
    contents = await file.read()
    return {
        "fileName": file.filename,
        "promptUsed": prompt,
        "extractedFields": {
            "vendor_name": "Demo Vendor",
            "total_amount": "123.45"
        },
        "rawResponse": {"source": "Colab mock"}
    }

# 🔁 Restart ngrok with correct port
port = 8000
public_url = ngrok.connect(port)
print("🚀 Your public Colab endpoint:", public_url)

# 🔧 Fix nested asyncio loop
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=port)


🚀 Your public Colab endpoint: NgrokTunnel: "https://3a21-34-87-137-1.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [628]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     196.117.160.61:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     196.117.160.61:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     196.117.160.61:0 - "GET /process HTTP/1.1" 405 Method Not Allowed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [628]


In [ ]:

class ColabInvoiceOCRProcessor:
    def __init__(self, model=None):
        self.model = model
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")

    def set_model(self, model):
        """Set your loaded model"""
        self.model = model
        print("Model loaded successfully!")

    def generate_prompt(self, selected_fields: List[str], output_format: str) -> str:
        """Generate dynamic prompt based on selected fields"""
        # Invoice field mappings
        field_descriptions = {
            "vendor_name": "Vendor/Supplier Name",
            "invoice_number": "Invoice Number",
            "invoice_date": "Invoice Date",
            "due_date": "Due Date",
            "total_amount": "Total Amount",
            "subtotal": "Subtotal",
            "tax_amount": "Tax Amount",
            "currency": "Currency",
            "billing_address": "Billing Address",
            "shipping_address": "Shipping Address",
            "line_items": "Line Items (products/services)",
            "payment_terms": "Payment Terms",
            "po_number": "Purchase Order Number",
            "customer_info": "Customer Information",
            "discount": "Discount Amount"
        }

        selected_descriptions = [field_descriptions[field] for field in selected_fields if field in field_descriptions]

        base_prompt = "Extract the following information from this invoice document:"
        field_prompt = "\\n".join([f"- {desc}" for desc in selected_descriptions])

        format_instructions = {
            "json": "Return the results in JSON format with field names as keys.",
            "csv": "Return the results in CSV format with headers.",
            "text": "Return the results in plain text format, one field per line.",
            "structured": "Return the results in a structured, readable format."
        }

        format_instruction = format_instructions.get(output_format, format_instructions["json"])

        # Complete prompt for your model
        full_prompt = f"""
        {base_prompt}

        {field_prompt}

        {format_instruction}

        Instructions:
        - If a field is not found, return "Not found" or null
        - For monetary amounts, include currency if visible
        - For dates, use MM/DD/YYYY format if possible
        - Be accurate and extract exactly what is shown
        - For line items, include product name, quantity, and price if available
        - Return only the requested information in the specified format
        """

        return full_prompt

    def process_with_model(self, image_data, prompt: str) -> Dict[str, Any]:
        """Process image with your loaded model"""
        try:
            if self.model is None:
                return {
                    "status": "error",
                    "message": "Model not loaded. Please load your model first."
                }

            # Convert image data to format your model expects
            if isinstance(image_data, str):
                # If it's a file path, load the image
                image = Image.open(image_data)
            elif isinstance(image_data, bytes):
                # If it's bytes, convert to PIL Image
                image = Image.open(io.BytesIO(image_data))
            else:
                # If it's already a PIL Image
                image = image_data

            # TODO: Replace this section with your actual model inference
            # Example for different model types:

            # For transformers models:
            # inputs = self.processor(images=image, text=prompt, return_tensors="pt")
            # outputs = self.model.generate(**inputs)
            # result = self.processor.decode(outputs[0], skip_special_tokens=True)

            # For other models:
            # result = self.model.predict(image, prompt)

            # PLACEHOLDER - Replace with your actual model inference
            result = self.mock_model_inference(image, prompt)

            return {
                "status": "success",
                "extracted_data": result,
                "confidence": 0.95,
                "processing_time": "1.2s"
            }

        except Exception as e:
            return {
                "status": "error",
                "message": f"Model processing failed: {str(e)}"
            }

    def mock_model_inference(self, image, prompt):
        """Mock inference - replace with your actual model call"""
        # This is just a placeholder - replace with your actual model
        return {
            "vendor_name": "Sample Vendor Corp",
            "invoice_number": "INV-2024-001",
            "invoice_date": "03/15/2024",
            "total_amount": "$1,234.56",
            "currency": "USD",
            "tax_amount": "$123.45"
        }

In [ ]:
# Install required packages in Colab
!pip install fastapi uvicorn python-multipart nest-asyncio pyngrok
!pip install Pillow pdf2image
!apt-get update && apt-get install -y poppler-utils



# Enable nested asyncio for Colab
nest_asyncio.apply()

# Global variable to store your model
YOUR_MODEL = None


# Initialize FastAPI app
app = FastAPI(title="Colab Invoice OCR API", version="1.0.0")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Configuration for Colab
UPLOAD_DIR = Path("/tmp/uploads")
TEMP_DIR = Path("/tmp/temp")
ALLOWED_EXTENSIONS = {'.png', '.jpg', '.jpeg', '.pdf'}
MAX_FILE_SIZE = 50 * 1024 * 1024  # 50MB for Colab

# Create directories
UPLOAD_DIR.mkdir(exist_ok=True)
TEMP_DIR.mkdir(exist_ok=True)

# Global processor instance
processor = ColabInvoiceOCRProcessor()

def validate_file(file: UploadFile) -> bool:
    """Validate uploaded file"""
    if not file.filename:
        return False

    file_ext = Path(file.filename).suffix.lower()
    if file_ext not in ALLOWED_EXTENSIONS:
        return False

    return True

def save_upload_file(upload_file: UploadFile) -> str:
    """Save uploaded file temporarily"""
    file_id = str(uuid.uuid4())
    file_ext = Path(upload_file.filename).suffix.lower()
    file_path = UPLOAD_DIR / f"{file_id}{file_ext}"

    with open(file_path, "wb") as buffer:
        shutil.copyfileobj(upload_file.file, buffer)

    return str(file_path)

def convert_pdf_to_images(pdf_path: str) -> List[str]:
    """Convert PDF to images for processing"""
    try:
        images = pdf2image.convert_from_path(pdf_path, dpi=200)
        image_paths = []

        for i, image in enumerate(images):
            image_path = TEMP_DIR / f"{Path(pdf_path).stem}_page_{i+1}.png"
            image.save(image_path, "PNG")
            image_paths.append(str(image_path))

        return image_paths
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"PDF conversion failed: {str(e)}")

def generate_preview(file_path: str) -> dict:
    """Generate preview for uploaded file"""
    try:
        file_ext = Path(file_path).suffix.lower()

        if file_ext == '.pdf':
            # Convert first page to image for preview
            images = pdf2image.convert_from_path(file_path, first_page=1, last_page=1, dpi=150)
            img_buffer = io.BytesIO()
            images[0].save(img_buffer, format='PNG')
            img_buffer.seek(0)

            preview_base64 = base64.b64encode(img_buffer.getvalue()).decode()
            total_pages = len(pdf2image.convert_from_path(file_path))

            return {
                "type": "pdf",
                "preview": f"data:image/png;base64,{preview_base64}",
                "pages": total_pages
            }
        else:
            # Handle image files
            with open(file_path, 'rb') as img_file:
                img_data = img_file.read()
                preview_base64 = base64.b64encode(img_data).decode()

            return {
                "type": "image",
                "preview": f"data:image/png;base64,{preview_base64}",
                "pages": 1
            }
    except Exception as e:
        return {"type": "error", "message": str(e)}

@app.get("/", response_class=HTMLResponse)
async def get_frontend():
    """Serve the Colab-optimized frontend"""
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Colab Invoice OCR Processor</title>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <style>
            body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; max-width: 1400px; margin: 0 auto; padding: 20px; background: #f5f5f5; }
            .header { text-align: center; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px; }
            .container { display: grid; grid-template-columns: 1fr 1fr; gap: 20px; }
            .card { background: white; border-radius: 10px; padding: 20px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
            .upload-area { border: 2px dashed #ccc; border-radius: 10px; padding: 20px; text-align: center; transition: all 0.3s; }
            .upload-area:hover { border-color: #667eea; background: #f8f9ff; }
            .preview-container { display: grid; grid-template-columns: repeat(auto-fill, minmax(200px, 1fr)); gap: 10px; margin-top: 15px; }
            .preview-item { border: 1px solid #ddd; border-radius: 8px; padding: 10px; background: #f9f9f9; }
            .preview-item img { max-width: 100%; height: auto; border-radius: 5px; }
            .fields-grid { display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px; margin: 15px 0; }
            .field-item { display: flex; align-items: center; padding: 8px; border: 1px solid #e0e0e0; border-radius: 5px; transition: all 0.2s; }
            .field-item:hover { background: #f0f8ff; border-color: #667eea; }
            .field-item input[type="checkbox"] { margin-right: 8px; }
            .btn { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 12px 24px; border: none; border-radius: 8px; cursor: pointer; font-size: 16px; transition: all 0.3s; }
            .btn:hover { transform: translateY(-2px); box-shadow: 0 4px 15px rgba(0,0,0,0.2); }
            .btn:disabled { opacity: 0.6; cursor: not-allowed; transform: none; }
            .output-section { grid-column: 1 / -1; margin-top: 20px; }
            .result-box { background: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #667eea; white-space: pre-wrap; font-family: 'Courier New', monospace; max-height: 400px; overflow-y: auto; }
            .loading { display: none; text-align: center; padding: 20px; }
            .loading-spinner { border: 4px solid #f3f3f3; border-top: 4px solid #667eea; border-radius: 50%; width: 40px; height: 40px; animation: spin 1s linear infinite; margin: 0 auto 10px; }
            @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
            .model-status { padding: 10px; border-radius: 5px; margin-bottom: 15px; }
            .model-loaded { background: #d4edda; color: #155724; border: 1px solid #c3e6cb; }
            .model-not-loaded { background: #f8d7da; color: #721c24; border: 1px solid #f5c6cb; }
            .select-all { margin-bottom: 15px; }
            .stats { display: grid; grid-template-columns: repeat(auto-fit, minmax(150px, 1fr)); gap: 10px; margin: 15px 0; }
            .stat-item { background: #f8f9fa; padding: 10px; border-radius: 5px; text-align: center; }
            .stat-value { font-size: 24px; font-weight: bold; color: #667eea; }
            .stat-label { font-size: 12px; color: #666; }
        </style>
    </head>
    <body>
        <div class="header">
            <h1>🔍 Colab Invoice OCR Processor</h1>
            <p>Upload invoices and extract structured data using your custom model</p>
        </div>

        <div id="modelStatus" class="model-status model-not-loaded">
            ⚠️ Model not loaded. Please load your model in Colab first.
        </div>

        <div class="container">
            <div class="card">
                <h2>📁 Upload Files</h2>
                <div class="upload-area" onclick="document.getElementById('fileInput').click()">
                    <p>🎯 Click to upload or drag & drop files</p>
                    <p style="color: #666; font-size: 14px;">Supported: PNG, JPG, PDF (Max 50MB)</p>
                </div>
                <input type="file" id="fileInput" multiple accept=".png,.jpg,.jpeg,.pdf" style="display: none;">

                <div class="stats" id="uploadStats" style="display: none;">
                    <div class="stat-item">
                        <div class="stat-value" id="fileCount">0</div>
                        <div class="stat-label">Files</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value" id="totalSize">0 MB</div>
                        <div class="stat-label">Total Size</div>
                    </div>
                </div>

                <div id="filePreview" class="preview-container"></div>
                <button onclick="uploadFiles()" class="btn" id="uploadBtn" style="width: 100%; margin-top: 15px;" disabled>
                    📤 Upload Files
                </button>
            </div>

            <div class="card">
                <h2>⚙️ Configuration</h2>

                <div class="select-all">
                    <button onclick="selectAllFields()" class="btn" style="background: #28a745; font-size: 14px; padding: 8px 16px;">
                        ✓ Select All Fields
                    </button>
                    <button onclick="clearAllFields()" class="btn" style="background: #dc3545; font-size: 14px; padding: 8px 16px; margin-left: 10px;">
                        ✗ Clear All
                    </button>
                </div>

                <h3>📋 Select Fields to Extract</h3>
                <div class="fields-grid" id="fieldsGrid"></div>

                <h3>📊 Output Format</h3>
                <select id="outputFormat" style="width: 100%; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                    <option value="json">JSON Format</option>
                    <option value="csv">CSV Format</option>
                    <option value="text">Plain Text</option>
                    <option value="structured">Structured Format</option>
                </select>

                <button onclick="processFiles()" class="btn" id="processBtn" style="width: 100%; margin-top: 20px;" disabled>
                    🚀 Process Files
                </button>

                <div class="loading" id="loading">
                    <div class="loading-spinner"></div>
                    <p>Processing with your model...</p>
                </div>
            </div>
        </div>

        <div class="output-section card">
            <h2>📊 Results</h2>
            <div id="results" class="result-box">Ready to process your invoices! Upload files and select fields to get started.</div>
        </div>

        <script>
            let uploadedFiles = [];
            let selectedFiles = [];

            const fields = {
                "vendor_name": "Vendor/Supplier Name",
                "invoice_number": "Invoice Number",
                "invoice_date": "Invoice Date",
                "due_date": "Due Date",
                "total_amount": "Total Amount",
                "subtotal": "Subtotal",
                "tax_amount": "Tax Amount",
                "currency": "Currency",
                "billing_address": "Billing Address",
                "shipping_address": "Shipping Address",
                "line_items": "Line Items",
                "payment_terms": "Payment Terms",
                "po_number": "Purchase Order Number",
                "customer_info": "Customer Information",
                "discount": "Discount Amount"
            };

            // Initialize fields
            const fieldsGrid = document.getElementById('fieldsGrid');
            for (const [key, label] of Object.entries(fields)) {
                const div = document.createElement('div');
                div.className = 'field-item';
                div.innerHTML = `
                    <input type="checkbox" name="fields" value="${key}" id="field_${key}">
                    <label for="field_${key}">${label}</label>
                `;
                fieldsGrid.appendChild(div);
            }

            // File input handler
            document.getElementById('fileInput').addEventListener('change', function(e) {
                selectedFiles = Array.from(e.target.files);
                updateFilePreview();
                document.getElementById('uploadBtn').disabled = selectedFiles.length === 0;
            });

            function updateFilePreview() {
                const previewDiv = document.getElementById('filePreview');
                const statsDiv = document.getElementById('uploadStats');

                if (selectedFiles.length === 0) {
                    previewDiv.innerHTML = '';
                    statsDiv.style.display = 'none';
                    return;
                }

                statsDiv.style.display = 'grid';
                document.getElementById('fileCount').textContent = selectedFiles.length;

                const totalSize = selectedFiles.reduce((sum, file) => sum + file.size, 0);
                document.getElementById('totalSize').textContent = (totalSize / 1024 / 1024).toFixed(2) + ' MB';

                previewDiv.innerHTML = '';
                selectedFiles.forEach((file, index) => {
                    const fileDiv = document.createElement('div');
                    fileDiv.className = 'preview-item';
                    fileDiv.innerHTML = `
                            <p><strong>${file.name}</strong></p>
                            <p>${(file.size / 1024 / 1024).toFixed(2)} MB</p>
                            <p>${file.type}</p>
                        `;
                    previewDiv.appendChild(fileDiv);
                });
            }

            function selectAllFields() {
                document.querySelectorAll('input[name="fields"]').forEach(cb => cb.checked = true);
            }

            function clearAllFields() {
                document.querySelectorAll('input[name="fields"]').forEach(cb => cb.checked = false);
            }

            async function uploadFiles() {
                const formData = new FormData();
                selectedFiles.forEach(file => formData.append('files', file));

                try {
                    document.getElementById('uploadBtn').disabled = true;
                    document.getElementById('uploadBtn').textContent = '📤 Uploading...';

                    const response = await fetch('/upload', {
                        method: 'POST',
                        body: formData
                    });

                    if (!response.ok) throw new Error('Upload failed');

                    const result = await response.json();
                    uploadedFiles = result.files;

                    // Update preview with server previews
                    const previewDiv = document.getElementById('filePreview');
                    previewDiv.innerHTML = '';

                    uploadedFiles.forEach(file => {
                        const fileDiv = document.createElement('div');
                        fileDiv.className = 'preview-item';
                        fileDiv.innerHTML = `
                            <p><strong>${file.filename}</strong></p>
                            <img src="${file.preview.preview}" style="max-width: 100%; height: auto; border-radius: 5px;">
                            <p>${file.preview.pages} page(s)</p>
                        `;
                        previewDiv.appendChild(fileDiv);
                    });

                    document.getElementById('processBtn').disabled = false;
                    document.getElementById('results').textContent = `✅ ${uploadedFiles.length} files uploaded successfully! Select fields and click Process.`;

                } catch (error) {
                    document.getElementById('results').textContent = `❌ Upload failed: ${error.message}`;
                } finally {
                    document.getElementById('uploadBtn').disabled = false;
                    document.getElementById('uploadBtn').textContent = '📤 Upload Files';
                }
            }

            async function processFiles() {
                const selectedFields = Array.from(document.querySelectorAll('input[name="fields"]:checked'))
                    .map(cb => cb.value);
                const outputFormat = document.getElementById('outputFormat').value;

                if (uploadedFiles.length === 0) {
                    alert('Please upload files first');
                    return;
                }

                if (selectedFields.length === 0) {
                    alert('Please select at least one field');
                    return;
                }

                document.getElementById('loading').style.display = 'block';
                document.getElementById('processBtn').disabled = true;

                try {
                    const response = await fetch('/process', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({
                            file_ids: uploadedFiles.map(f => f.file_id),
                            selected_fields: selectedFields,
                            output_format: outputFormat
                        })
                    });

                    if (!response.ok) throw new Error('Processing failed');

                    const result = await response.json();
                    document.getElementById('results').textContent = JSON.stringify(result, null, 2);

                } catch (error) {
                    document.getElementById('results').textContent = `❌ Processing failed: ${error.message}`;
                } finally {
                    document.getElementById('loading').style.display = 'none';
                    document.getElementById('processBtn').disabled = false;
                }
            }

            // Check model status periodically
            async function checkModelStatus() {
                try {
                    const response = await fetch('/model-status');
                    const status = await response.json();
                    const statusDiv = document.getElementById('modelStatus');

                    if (status.loaded) {
                        statusDiv.className = 'model-status model-loaded';
                        statusDiv.innerHTML = '✅ Model loaded and ready!';
                    } else {
                        statusDiv.className = 'model-status model-not-loaded';
                        statusDiv.innerHTML = '⚠️ Model not loaded. Please load your model in Colab first.';
                    }
                } catch (error) {
                    console.error('Error checking model status:', error);
                }
            }

            // Check model status on page load and every 5 seconds
            checkModelStatus();
            setInterval(checkModelStatus, 5000);
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)

@app.post("/upload")
async def upload_files(files: List[UploadFile] = File(...)):
    """Upload and process multiple files"""
    if not files:
        raise HTTPException(status_code=400, detail="No files uploaded")

    uploaded_files = []

    for file in files:
        if not validate_file(file):
            raise HTTPException(status_code=400, detail=f"Invalid file type: {file.filename}")

        file_path = save_upload_file(file)
        file_id = Path(file_path).stem
        preview = generate_preview(file_path)

        uploaded_files.append({
            "file_id": file_id,
            "filename": file.filename,
            "file_path": file_path,
            "preview": preview
        })

    return {"status": "success", "files": uploaded_files}

@app.post("/process")
async def process_files(request: Request):
    """Process uploaded files with your model"""
    try:
        data = await request.json()
        file_ids = data.get("file_ids", [])
        selected_fields = data.get("selected_fields", [])
        output_format = data.get("output_format", "json")

        if not file_ids:
            raise HTTPException(status_code=400, detail="No files specified")

        if not selected_fields:
            raise HTTPException(status_code=400, detail="No fields selected")

        # Generate prompt
        prompt = processor.generate_prompt(selected_fields, output_format)

        results = []

        for file_id in file_ids:
            # Find file path
            file_path = None
            for path in UPLOAD_DIR.glob(f"{file_id}.*"):
                file_path = str(path)
                break

            if not file_path:
                continue

            # Process file
            if file_path.lower().endswith('.pdf'):
                # Convert PDF to images and process each page
                image_paths = convert_pdf_to_images(file_path)
                for i, img_path in enumerate(image_paths):
                    result = processor.process_with_model(img_path, prompt)
                    result["file_id"] = file_id
                    result["page"] = i + 1
                    results.append(result)
            else:
                # Process image directly
                result = processor.process_with_model(file_path, prompt)
                result["file_id"] = file_id
                result["page"] = 1
                results.append(result)

        return {
            "status": "success",
            "results": results,
            "selected_fields": selected_fields,
            "output_format": output_format,
            "processed_at": datetime.now().isoformat(),
            "prompt_used": prompt  # Show the generated prompt in results
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/model-status")
async def get_model_status():
    """Check if model is loaded"""
    return {
        "loaded": processor.model is not None,
        "model_type": type(processor.model).__name__ if processor.model else None
    }

@app.post("/load-model")
async def load_model(request: Request):
    """Load model from Colab (call this from your notebook)"""
    try:
        data = await request.json()
        model = data.get("model")
        processor.set_model(model)
        return {"status": "success", "message": "Model loaded successfully"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

@app.get("/fields")
async def get_available_fields():
    """Get available invoice fields"""
    return {
        "fields": {
            "vendor_name": "Vendor/Supplier Name",
            "invoice_number": "Invoice Number",
            "invoice_date": "Invoice Date",
            "due_date": "Due Date",
            "total_amount": "Total Amount",
            "subtotal": "Subtotal",
            "tax_amount": "Tax Amount",
            "currency": "Currency",
            "billing_address": "Billing Address",
            "shipping_address": "Shipping Address",
            "line_items": "Line Items",
            "payment_terms": "Payment Terms",
            "po_number": "Purchase Order Number",
            "customer_info": "Customer Information",
            "discount": "Discount Amount"
        }
    }

@app.delete("/cleanup")
async def cleanup_files():
    """Clean up temporary files"""
    try:
        for file_path in UPLOAD_DIR.glob("*"):
            file_path.unlink()
        for file_path in TEMP_DIR.glob("*"):
            file_path.unlink()
        return {"status": "success", "message": "Files cleaned up"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

# Function to load your model into the processor
def load_your_model(model):
    """Call this function from your Colab notebook to load your model"""
    global processor
    processor.set_model(model)
    print("✅ Model loaded into OCR processor!")

# Function to start the server
def start_server(port=8000, use_ngrok=False):
    """Start the FastAPI server"""
    if use_ngrok:
        # Install and setup ngrok for public access
        try:
            from pyngrok import ngrok
            public_url = ngrok.connect(port)
            print(f"🌐 Public URL: {public_url}")
        except ImportError:
            print("⚠️ pyngrok not installed. Install with: !pip install pyngrok")
            use_ngrok = False

    # Run the FastAPI app with uvicorn
    # Use a separate thread to avoid blocking the Colab notebook
    def run_server():
        uvicorn.run(app, host="0.0.0.0", port=port)

    thread = Thread(target=run_server)
    thread.daemon = True  # Allow the thread to exit when the main program exits
    thread.start()

    print(f"🎉 FastAPI server started on port {port}")
    if not use_ngrok:
        print("✨ To access the UI, use the local URL provided by uvicorn output.")

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading